LlamaIndex 기반 심플한 Naive-RAG 구성해보기

- 활용 Dataset: Huggingface 코리안 웹텍스트 데이터셋 (https://huggingface.co/datasets/HAERAE-HUB/KOREAN-WEBTEXT)
- * 인덱스 0~20까지.
- 임베딩 모델: text-embedding-3-small
- Generation 모델: gpt-4o-mini
- 벡터DB : Qdrant
- 노드(청크) 단위: 사이즈 500, 50글자씩 겹치게
- 양자화 config: SQ(Scalar Quantization)
- Query 엔진 연결시켜서 Q&A 사이클 완성시키기


In [ ]:
!pip install llama-index-vector-stores-qdrant==0.2.10
!pip install datasets

In [ ]:
!pip install llama-index==0.10.58
!pip install openai==1.37.1
!pip install qdrant-client==1.10.0
!pip install pandas==2.2.2
!pip install gensim==4.3.3
!pip install nest-asnycio==1.6.0

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import os, json
key_config = json.load(open("/content/drive/MyDrive/fastcampus/RAG/key_config.json", "r"))

In [55]:
key_config.keys()

dict_keys(['gpt_key', 'qdrant_key', 'qdrant_url', 'cohere_key', 'tavily_key'])

In [56]:
# OpenAI 키 설정
import os, json
os.environ["OPENAI_API_KEY"] = key_config['gpt_key']
qdrant_key = key_config['qdrant_key']
qdrant_url = key_config['qdrant_url']

# 1. 원천 데이터 -> LlamaIndex Document Object까지 전처리

In [10]:
from llama_index.core import Document, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
import nest_asyncio
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext
import qdrant_client
from qdrant_client import models
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline
nest_asyncio.apply()


In [11]:
# Dataset 로드

# from datasets import load_dataset

# https://huggingface.co/datasets/HAERAE-HUB/KOREAN-WEBTEXT
# ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
# Pandas DataFrame 오브젝트로 변환
# data = ds.to_pandas()[['text', 'source','token_count', '__index_level_0__']]
# data.head()

import pandas as pd
data = pd.read_csv('/content/korean_webtext.csv')

In [12]:
data.head()

,Unnamed: 0,text,source,token_count,__index_level_0__
0,0,사이트의 판매량에 기반하여 판매량 추이를 반영한 인터파크 도서에서의 독립적인 판매 ...,oscar2201,3348,0
1,1,“아~아~잊으랴 어찌 우리 이날을 조국의 원수들이 짓밟아 오던 날을~”6·25의 노...,oscar2201,1427,1
2,2,일러전쟁의 승패를 가른 쓰시마 해전은 세계 최강으로 평가 받는 발틱함대를 괴멸시켰다...,oscar2201,2458,2
3,3,"재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 ...",oscar2201,2838,3
4,4,"상급자의 범죄와 비리, 부패를 하급자에게 돌리는 것으로 따지면 타의추종을 불허하는 ...",oscar2201,1628,4


In [13]:
# Document 오브젝트로 변환
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        extra_info={'__index_level_0__':row['__index_level_0__']}
    ))

In [ ]:
docs

In [15]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# Document 오브젝트 Transformation을 위한 파이프라인 오브젝트 생성 - 요건: 청크사이즈 500, 오버랩 50
# document_title: TitleExtractor를 활용해 문서 내용을 바탕으로 LLM 기반의 title 생성
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=500, chunk_overlap=50),
        TitleExtractor(llm=OpenAI(model="gpt-4o-mini")),
        OpenAIEmbedding(model="text-embedding-3-small")
    ]
)

In [16]:
# 판다스 DataFrame -> 라마인덱스 다큐먼트 오브젝트 -> 노드 오브젝트로 변환
nodes = pipeline.run(documents=docs)

100%|██████████| 5/5 [00:01<00:00,  3.16it/s]


In [19]:
len(nodes)

178

In [59]:
nodes[0].metadata

{'__index_level_0__': 0,
 'document_title': '"앨런 홀링허스트의 \'수영장 도서관\': 퀴어 문학의 혁신과 귀족 정체성 탐구"'}

# 2. Qdrant 벡터스토어 셋업 및 데이터 인덱싱

In [20]:
# 사용 LLM 설정 - Generation 모델: gpt-4o-mini, 임베딩 모델: text-embedding-3-small
Settings.llm = OpenAI(model="gpt-4o-mini", temperature=0.1)
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)

In [57]:
# Qdrant 클라우드 DB 연결
client = qdrant_client.QdrantClient(
    url=qdrant_url,
    api_key=qdrant_key,
)

In [25]:
# DB 내 실습용 콜렉션 생성 및 양자화 config 설정 - 요건: Scalar Quantization, 99퍼센타일
client.recreate_collection(
    collection_name="subject_test",
    vectors_config=models.VectorParams(
        size=1536,
        distance=models.Distance.COSINE
    ),
    quantization_config=models.ScalarQuantization(
        scalar=models.ScalarQuantizationConfig(
            type=models.ScalarType.INT8,
            quantile=0.99,
            always_ram=True,
        )
    ),
)

<ipython-input-25-572cadfd27b7>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [26]:
from llama_index.core import StorageContext

# VectorstoreIndex의 Backend로써 storage_context 부여 및 인덱싱
vector_store = QdrantVectorStore(client=client, collection_name="subject_test") # 쿼드란트 벡터스토어 만들기
storage_context = StorageContext.from_defaults(vector_store=vector_store) # 스토리지 컨텍스트로써 쿼드란트 벡터스토어 연결시키기

# 벡터스토어 인덱스의 백엔드로 최종 연결성 맺기
# 1. 노드 방식
index = VectorStoreIndex(nodes, storage_context=storage_context)

# 2. 도큐먼트 방식
# index = VectorStoreIndex.from_documents(docs, storage_context=storage_context)

# 3.쿼리 엔진으로 인덱스 연결 및 샘플 쿼리 몇개 날려보는 것으로 Naive-RAG 마무리

In [27]:
# 인덱스 쿼리엔진에 연결
query_engine = index.as_query_engine()

In [37]:
data

,Unnamed: 0,text,source,token_count,__index_level_0__
0,0,사이트의 판매량에 기반하여 판매량 추이를 반영한 인터파크 도서에서의 독립적인 판매 ...,oscar2201,3348,0
1,1,“아~아~잊으랴 어찌 우리 이날을 조국의 원수들이 짓밟아 오던 날을~”6·25의 노...,oscar2201,1427,1
2,2,일러전쟁의 승패를 가른 쓰시마 해전은 세계 최강으로 평가 받는 발틱함대를 괴멸시켰다...,oscar2201,2458,2
3,3,"재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 ...",oscar2201,2838,3
4,4,"상급자의 범죄와 비리, 부패를 하급자에게 돌리는 것으로 따지면 타의추종을 불허하는 ...",oscar2201,1628,4
5,5,최근 언론보도에 의하면 이재현 CJ그룹 회장이 지난해 말 두 자녀에게 증여하였던 주...,oscar2201,1366,5
6,6,"나는 노무현의 시대를 살지 않았다. 그러니까, 나는 이 땅의 생명체로 살아있긴 했지...",oscar2201,2017,6
7,7,CBRE가 21일 발표한 ‘2021년 2분기 국내 상업용 부동산 시장 보고서’에 따...,oscar2201,1421,7
8,8,"안녕하세요. 한화솔루션입니다. 지난주, 슬기로운 솔루션 직장생활 2탄에 이어 이번엔...",oscar2201,2143,8
9,9,캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이틀...,oscar2201,1104,9


In [60]:
#쿼리 날려보기
response = query_engine.query("재테크 채널 유튜버는 무엇을 했어?")
print(response)

재테크 채널 유튜버는 통장 관리부터 채권 투자까지 다양한 돈 관리 방법을 소개하며, 경제적 자유를 얻을 수 있도록 알찬 재테크 정보를 쉽고 재미있게 전달하는 칼럼 시리즈를 운영하고 있습니다.


In [76]:
node = query_engine.retrieve("재테크 채널 유튜버는 무엇을 했어?")

In [77]:
# 질문에 해당하는 노드 찾기
node[0]

NodeWithScore(node=TextNode(id_='c62f352d-744b-4050-a28c-85c8b0bf078c', embedding=None, metadata={'__index_level_0__': 3, 'document_title': '"재테크와 월급 관리: 경제적 자유를 위한 통장 관리 및 비상예비자금 전략"'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7562ebec-1950-4f17-89fb-36ad396f756f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'__index_level_0__': 3}, hash='536c81b7151754babba3664e00cf538e916abf9aa010c58c1b9d8a663fb2edd3'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='3de2534d-0d53-4104-bced-ca2135af5521', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='51713a89f010022782ae7d6fb46ad141a42d06895f09e4313718186a80e5444f')}, text='재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 오상열의 <어차피 재테크는 토스> 칼럼 시리즈에서는, 재테크의 가장 기본인 통장 관리부터 채권 투자까지 전반적인 돈 관리 방법을 하나씩 짚게 됩니다. 모두가 경제적 자유를 얻을 수 있도록, 알찬 재테크 정보를 쉽고 재미있게 전달합니다. 월급이 들어오는 통장 관리, 어떻게 하고 계시나요? 월급이란 근로와 사업의 댓가로 매월 통장으로 들어오는 수입을 말합니다. 이 수입을 가지고 생활비로도 쓰고, 저축

In [68]:
#쿼리 날려보기
response = query_engine.query("한국한의학연구원의 유튜브 채널에는 어떤게 있어?")
print(response)

한국한의학연구원의 유튜브 채널에는 김종열 원장의 사상체질에 대한 강연이 소개되어 있습니다.


In [72]:
node = query_engine.retrieve("한국한의학연구원의 유튜브 채널에는 어떤게 있어?")

In [74]:
# 질문에 해당하는 노드 찾기
node[0]

NodeWithScore(node=TextNode(id_='85088dce-3c6d-441d-b189-c6d97964d470', embedding=None, metadata={'__index_level_0__': 18, 'document_title': '"Exploring the Reliability and Diversity of Constitutional Theories in Traditional Medicine: A Comparative Study of Sasang, Eight Constitutions, and Beyond"'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5cfa4353-dbe8-4a8f-8ba5-32a57f990bf5', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'__index_level_0__': 18}, hash='b577bd903c578e2d64ca88bec85bcc942c466a489180cdc08636f30906a7c998'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5fe73568-bcae-4d3f-8ded-2ae41544eee7', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='aa88341aa3bb2b2e721e5563a807513a867286c0e471e2d44db5bb3b850444ea')}, text="한국한의학연구원의 유튜브 채널에 최근 사상체질에 대한 김종열 한국한의학연구원장의 강연이 소개됐다. 그는 원장으로 취임하기 전에는 한국한의학연구원에서 주로 사상체질 및 체질진단기기를 연구했으며 그 공로로 과학기술훈장 진보장을 수상하기도 했다. 사상체질의학은 조선말 이제마가 혼